<h1 style="color:blue;">Group 4 Model Sandbox</h1>


https://mesonet.agron.iastate.edu/request/download.phtml?network=NJ_ASOS

<font size="5">Members:</font> <br> Birendra Khimding <br> Muzhgan Rustaqi <br> Andrew Fennimore

#### Used to help determine models and best fit

In [111]:
import pandas as pd
import numpy as np


# Pull data from MJX
url = "https://raw.githubusercontent.com/Fenn3963/Weather-Impact-on-Air-Traffic-Management/refs/heads/main/MJX.csv"

#All values with na are labeled as M
weather = pd.read_csv("MJX.csv" , na_values= "M")

#Remove all of the variables that have less then a 15% threshhold
threshold = len(weather)*.15
cols_drop_nan = weather.columns[weather.isna().sum() <= threshold]

# Drop row with missing values
weather.dropna(subset=cols_drop_nan, inplace=True)


cols_to_drop = weather.columns[weather.isna().sum() > 0]

weather.drop(columns=cols_to_drop , inplace=True)



In [113]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Read Data from the MJX
url = "https://raw.githubusercontent.com/Fenn3963/Weather-Impact-on-Air-Traffic-Management/main/MJX.csv"

# All values with na are labeled as M
weather = pd.read_csv(url, na_values="M")



# Model 1: Predict visibility using various weather variables
model = smf.ols(formula="vsby ~ tmpf + relh + sknt + alti + C(skyc1)" , 
                data=weather).fit()

# Print summary of the model
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                   vsby   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     421.0
Date:                Sun, 23 Feb 2025   Prob (F-statistic):               0.00
Time:                        14:51:22   Log-Likelihood:                -25046.
No. Observations:               11267   AIC:                         5.011e+04
Df Residuals:                   11257   BIC:                         5.019e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.1464      3.153     

In [114]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# pull data from 
url = "https://raw.githubusercontent.com/Fenn3963/Weather-Impact-on-Air-Traffic-Management/main/MJX.csv"

# All values with na are labeled as M
weather = pd.read_csv(url, na_values="M")



# Model 1: Predict visibility using various weather variables
model = smf.ols(formula="vsby ~ tmpf + relh + sknt + alti + C(skyc1)", data=weather).fit()

# Print summary of the model
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                   vsby   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     421.0
Date:                Sun, 23 Feb 2025   Prob (F-statistic):               0.00
Time:                        14:51:23   Log-Likelihood:                -25046.
No. Observations:               11267   AIC:                         5.011e+04
Df Residuals:                   11257   BIC:                         5.019e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.1464      3.153     

In [116]:
#Inversing and transforming to deal with negative skew
import numpy as np
import statsmodels.formula.api as smf

# tyring an inverse log tranformation
K = weather["vsby"].max() + 1
weather["reflect_log_vsby"] = np.log(K - weather["vsby"])

# Fit the ols model using the new variable
model = smf.ols(formula="reflect_log_vsby ~ tmpf + relh + sknt + alti + C(skyc1)", data=weather).fit()

# View the summary of the model
print(model.summary())



                            OLS Regression Results                            
Dep. Variable:       reflect_log_vsby   R-squared:                       0.276
Model:                            OLS   Adj. R-squared:                  0.275
Method:                 Least Squares   F-statistic:                     475.6
Date:                Sun, 23 Feb 2025   Prob (F-statistic):               0.00
Time:                        14:51:23   Log-Likelihood:                -10746.
No. Observations:               11267   AIC:                         2.151e+04
Df Residuals:                   11257   BIC:                         2.159e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           3.9165      0.886     

In [119]:
#Transform Y so that it is reflected and reversed
# constant K greater than the maximum value of vsby
K = weather['vsby'].max() + 1

# Apply the reverse log transformation and create the GLM model
modelglm_neg = smf.glm(formula=f"np.log({K} - vsby) ~ tmpf + dwpf + relh + drct + sknt + alti + mslp + gust + \
                        (skyl1 * skyl2 * skyl3) + \
                        peak_wind_gust + \
                        feel", data=weather).fit()

print(modelglm_neg.summary())


                  Generalized Linear Model Regression Results                  
Dep. Variable:     np.log(11.0 - vsby)   No. Observations:                   18
Model:                             GLM   Df Residuals:                        1
Model Family:                 Gaussian   Df Model:                           16
Link Function:                Identity   Scale:                      1.5883e-08
Method:                           IRLS   Log-Likelihood:                 14.240
Date:                 Sun, 23 Feb 2025   Deviance:                      0.21659
Time:                         14:51:25   Pearson chi2:                    0.217
No. Iterations:                      3   Pseudo R-squ. (CS):              1.000
Covariance Type:             nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            53.5564

C:\Users\fenn3\anaconda3\Lib\site-packages\statsmodels\regression\_tools.py:121: RuntimeWarning: divide by zero encountered in scalar divide
  scale = np.dot(wresid, wresid) / df_resid


In [121]:
#Seeing if adding mean changes results
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Read Data from the MJX
url = "https://raw.githubusercontent.com/Fenn3963/Weather-Impact-on-Air-Traffic-Management/main/MJX.csv"

# All values with na are labeled as M
weather = pd.read_csv(url, na_values="M")

# List of significant variables to fill missing values with their mean
significant_variables = ["vsby", "tmpf", "dwpf", "relh", "drct", "skyl1", "skyl3", "feel", "mslp"]

# Fill missing values with the mean for specified significant variables
for variable in significant_variables:
    weather[variable].fillna(weather[variable].mean(), inplace=True)

# Create a new model using only significant variables
model_sig = smf.ols(formula="vsby ~ tmpf + relh + drct + skyl1 + skyl3 + feel + mslp", 
                    data=weather).fit()

# Print summary of the new model
print(model_sig.summary())


                            OLS Regression Results                            
Dep. Variable:                   vsby   R-squared:                       0.248
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                     538.7
Date:                Sun, 23 Feb 2025   Prob (F-statistic):               0.00
Time:                        14:51:27   Log-Likelihood:                -25461.
No. Observations:               11423   AIC:                         5.094e+04
Df Residuals:                   11415   BIC:                         5.100e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -25.1164      3.414     -7.356      0.0

C:\Users\fenn3\AppData\Local\Temp\ipykernel_25668\69535338.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather[variable].fillna(weather[variable].mean(), inplace=True)


In [135]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error

K = weather['vsby'].max() + 1

# Calculate the log-transformed visibility
weather['log_transformed_vsby'] = np.log(K - weather['vsby'])


# Fit the glm model with log link
model_gamma = smf.glm(formula="log_transformed_vsby ~ tmpf + relh + C(skyc1) + sknt + alti", 
                      data=weather,
                      family=sm.families.Gamma(link=sm.families.links.log())).fit()

# summary of model
print(model_gamma.summary())

weather['predictions'] = model_gamma.predict(weather)

mae = mean_absolute_error(weather['log_transformed_vsby'], weather['predictions'])



C:\Users\fenn3\anaconda3\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                  Generalized Linear Model Regression Results                   
Dep. Variable:     log_transformed_vsby   No. Observations:                11275
Model:                              GLM   Df Residuals:                    11265
Model Family:                     Gamma   Df Model:                            9
Link Function:                      log   Scale:                          102.68
Method:                            IRLS   Log-Likelihood:                    inf
Date:                  Sun, 23 Feb 2025   Deviance:                   6.5731e+05
Time:                          15:20:01   Pearson chi2:                 1.16e+06
No. Iterations:                     100   Pseudo R-squ. (CS):                nan
Covariance Type:              nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          20.

C:\Users\fenn3\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:812: RuntimeWarning: divide by zero encountered in log
  ll_obs -= special.gammaln(weight_scale) + np.log(endog)
C:\Users\fenn3\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1891: RuntimeWarning: invalid value encountered in scalar subtract
  prsq = 1 - np.exp((self.llnull - self.llf) * (2 / self.nobs))


ValueError: Input contains NaN.